In [2]:
%load_ext autoreload
%autoreload 2
    
!pip install pandas_gbq
import numpy as np
import pandas as pd
import pandas_gbq as bq
import re
import json
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from google.oauth2 import service_account
pd.options.display.float_format = "{:.4f}".format
from mmm_eval.comparison.load import get_datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from mmm_eval.comparison.load import get_datasets
stage = "warchest-staging-323804"  # "warchest-develop"
import os

# customer_id = "belong_64"
# dataset_name = customer_id + '_datamart'
# data_version = "2025-07-10T05:08:31.208146"

customer_id = "belong_64"
dataset_name = customer_id + '_datamart'
data_version = "2025-07-07T05:59:08.517992"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/samuelmccormick/.config/gcloud/application_default_credentials.json"
project_id = "mtx-dataos-datalake-prod"

customer_name = customer_id.rpartition("_")[0]
datasets = get_datasets(project_id, dataset_name, data_version, node_filter="default_default_internet")

/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-07-22 10:20:30,224 - mmm_eval.comparison.load - INFO - Loading discounts_snapshot
2025-07-22 10:20:34,244 - mmm_eval.comparison.load - INFO - Loaded discounts_snapshot with 7536 rows
/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-07-22 10:20:35,365 - mmm_eval.comparison.load - INFO - Loading events_snapshot
2025-07-22 10:20:37,491 - mmm_eval.comparison.load - INFO - Loaded events_snapshot with 8 rows
/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.

## Design

Pass the following args:

- Customer slug
- Data version
- Holiday names
- Pipeline link

then

- process all datasets and merge them together
- call functions to construct data configs for each framework
- transformations
  - PyMC: maxabs scale all control columns
  - Meridian: add dummy geo column

## Turn into DFs for PyMC
- Restrict to node(s) of interest
- Convert all datasets to weekly
- Load holidays and process them
- Merge together into a single dataframe
- Set up config fields based on dataset column mapping

In [3]:
import mmm_eval.comparison.utils as utils
from mmm_eval.comparison.load import aggregate_to_node_level, convert_df_to_weekly

# sales_weekly = convert_df_to_weekly(
#     datasets["sales_snapshot"],
#     numerical_columns=["quantity", "value"],
#     downsample_method_to_daily={"value": utils.DownsampleMethod.UNIFORM, "quantity": utils.DownsampleMethod.UNIFORM},
#     agg_method_to_weekly={"value": "sum", "quantity": "sum"},
# )

# sales = aggregate_to_node_level(
#     sales_weekly, extra_group_cols=[], agg_mapping={"quantity": "sum", "value": "sum"}
# )

# sales_proc = sales[["date", "quantity", "value"]].set_index("date").rename(columns={"value": "revenue"})
# sales_proc.head()

In [4]:
# drop_columns = [
#     "impressions",
#     "funnel_stage",
#     "tarp",
#     "fees",
# ]

# if "impressions" in datasets["paid_media_snapshot"].columns:
#     datasets['paid_media_snapshot'].drop(columns=drop_columns, inplace=True)

# # Convert to weekly then aggregate to node or lower level
# paid_media_weekly = convert_df_to_weekly(
#     datasets['paid_media_snapshot'],
#     numerical_columns=["spend"],
#     downsample_method_to_daily={
#         "spend": utils.DownsampleMethod.UNIFORM,
#     },
#     agg_method_to_weekly={
#         "spend": "sum",
#     },
# )

# paid_media = aggregate_to_node_level(
#     paid_media_weekly, extra_group_cols=["media_channel", "marketing_spend_impact"], agg_mapping={"spend": "sum"}
# )

In [5]:
# pivot out by spend impact
# pivoted = paid_media.pivot(columns=["media_channel", "marketing_spend_impact"], values="spend", index="date")
# pivoted.columns = ['_'.join(col).strip() for col in pivoted.columns.values]
# paid_media_pivoted = pivoted.fillna(0)
# paid_media_pivoted.head()

In [6]:
# pricing_weekly = convert_df_to_weekly(
#     datasets["pricing_snapshot"],
#     numerical_columns=["retail_price"],
#     downsample_method_to_daily={
#         "retail_price": utils.DownsampleMethod.REPLICA,
#     },
#     agg_method_to_weekly={
#         "retail_price": "mean",
#     },
# )

# pricing = aggregate_to_node_level(
#     pricing_weekly, extra_group_cols=["company"], agg_mapping={"retail_price": "mean"}
# )
# # alternative is to include competitor prices as separate cols
# pricing = pricing[pricing["company"]=="bank_australia"]
# pricing = pricing[["date", "retail_price"]].set_index("date").ffill()
# pricing.head()

In [7]:
offers_weekly = convert_df_to_weekly(
    datasets["offers_snapshot"],
    numerical_columns=["offer_value", "offer_measure"],
    downsample_method_to_daily={
        "retail_price": utils.DownsampleMethod.UNIFORM,
    },
    agg_method_to_weekly={
        "offer_measure": "sum",
        "offer_value": "sum",
    },
)

offers = aggregate_to_node_level(
    offers_weekly, extra_group_cols=["company_name"], agg_mapping={
            "offer_measure": "sum",
            "offer_value": "sum",
        },
)
# alternative is to include competitor prices as separate cols
offers = offers[offers["company_name"].isin(["belong", "default"])]
value_col = "offer_measure" if offers["offer_value"].isnull().all() else "offer_value"
    
offers = offers[["date", value_col]].set_index("date").ffill()
offers.head()

/Users/samuelmccormick/work/mmm-eval/mmm_eval/comparison/load.py:360: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


,offer_value
date,
2021-08-09,360.0000
2021-08-16,420.0000
2021-08-23,420.0000
2021-08-30,420.0000
2021-09-06,420.0000


In [8]:
# events_weekly = convert_df_to_weekly(
#     datasets["events_snapshot"],
#     numerical_columns=["duration",],
#     downsample_method_to_daily=utils.DownsampleMethod.REPLICA,
#     agg_method_to_weekly={
#         "duration": "sum",
#     },
# )

# events = aggregate_to_node_level(
#     events_weekly, extra_group_cols=["event_type", "event_purpose"], agg_mapping={"duration": "sum"}
# )
# events

In [9]:
# binary_matrix = pd.crosstab(events["date"], events["event_purpose"])
# binary_matrix = binary_matrix.astype(bool).astype(int)
# events_dense = binary_matrix.reindex(sales_proc.index, fill_value=0)
# events_dense.head()

In [10]:
# merged = pd.merge(sales_proc, paid_media_pivoted, left_index=True, right_index=True)
# merged = pd.merge(merged, pricing, left_index=True, right_index=True)
# merged = pd.merge(merged, events_dense, left_index=True, right_index=True)
# merged.head()

# Fetch relevant holidays

In [11]:
# from random pipeline
#holidays = pd.read_parquet("gs://mtx-ths-uat-au-theseus-pipeline-artifacts/artifacts/816012989105/theseus-simply-energy-20250709103055/run-feature-store-2_-5522238876388687872/model_output/export/holidays.parquet")
#holidays.head()

,period_start,variable,value
0,2009-12-28,ANZAC Day,0.0000
1,2010-01-04,ANZAC Day,0.0000
2,2010-01-11,ANZAC Day,0.0000
3,2010-01-18,ANZAC Day,0.0000
4,2010-01-25,ANZAC Day,0.0000


In [12]:
# model_holidays = ["Christmas Day", "Good Friday", "New Year's Day"]
# holidays = holidays[(holidays["variable"].isin(model_holidays))].rename(columns={"period_start": "date"})
# holidays = holidays.pivot(index="date", columns="variable", values="value")

# merged = pd.merge(merged, holidays, left_index=True, right_index=True)
# merged

In [16]:
#pd.read_parquet("gs://mtx-ths-dev-us-theseus-pipeline-artifacts/artifacts/757046730850/theseus-xero-20250715122215/run-feature-store-2_-6097048265306406912/model_output/export/externals.parquet")

In [21]:
import os

#os.listdir("../data")
chicken = pd.read_parquet("../data/chicken_treat.parquet")

In [25]:
chicken["cpi_trimmed_mean_feat_smoothed"]

date
2019-12-30   0.0010
2020-01-06   0.0010
2020-01-13   0.0010
2020-01-20   0.0010
2020-01-27   0.0010
              ...  
2025-04-21   0.0010
2025-04-28   0.0010
2025-05-05   0.0010
2025-05-12   0.0010
2025-05-19   0.0010
Name: cpi_trimmed_mean_feat_smoothed, Length: 282, dtype: Float64

In [26]:
from mmm_eval.comparison.process import load_and_process_datasets

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/samuelmccormick/.config/gcloud/application_default_credentials.json"

# belong.default.internet
#customer_id = "belong_64"
#data_version = "2025-07-07T05:59:08.517992"
#pipeline_path = "gs://mtx-ths-uat-au-theseus-pipeline-artifacts/artifacts/816012989105/theseus-belong-20250714150222/run-feature-store-2_490770313592700928/model_output/export"

# xero (default.smb.smb)
customer_id = "xero_51"
data_version = "2025-07-09T06:24:18.236648"
pipeline_path = "gs://mtx-ths-dev-us-theseus-pipeline-artifacts/artifacts/757046730850/theseus-xero-20250715122215/run-feature-store-2_-6097048265306406912/model_output/export"

# chicken_treat (default.consideration.at_store)
#customer_id = "chicken_treat_53"
#data_version = "2025-06-25T00:18:48.379609"
#pipeline_path = "gs://mtx-ths-uat-us-theseus-pipeline-artifacts/artifacts/816012989105/theseus-chicken-treat-20250627150301/run-feature-store-2_5949443570921373696/model_output/export"

# original (default.core.gas)


holidays_whitelist = [
  'BAS Deadline (Q1)',
  'BAS Deadline (Q2)',
  'BAS Deadline (Q3)',
  'BAS Deadline (Q4)',
  'Christmas Day',
  'EOFY',
  'Lockdown Start',
  'Lodgement Deadline',
  'National Day of Mourning for Queen Elizabeth II',
  "New Year's Day",
]
# why are some of these not showing up?
externals_whitelist = [
  'cpi_trimmed_mean_feat_smoothed',
  'consumer_sentiment_feat_centered',
  'employed_total_persons_seasonally_adjusted_feat_differenced',
  'target_cash_rate_rolling_mean',
                      ]
#node_filter = "default_consideration_at_store"
datasets_processed = load_and_process_datasets(customer_id, data_version, pipeline_path,
                                               holidays_whitelist, externals_whitelist) # node_filter=node_filter)

2025-07-16 10:57:34,914 - mmm_eval.comparison.process - INFO - Loading datasets for customer xero_51 with data version 2025-07-09T06:24:18.236648 and node filter None
/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-07-16 10:57:38,527 - mmm_eval.comparison.load - INFO - Loading earned_media_snapshot
2025-07-16 10:57:39,291 - mmm_eval.comparison.load - INFO - Loaded earned_media_snapshot with 230 rows
/Users/samuelmccormick/work/mmm-eval/.venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-07-16 10:57:40,077 - mmm_eval.comparison.load - WARNING - events_snapshot was not found in datamart
2025-07-16 10:57:41,028 - mmm_eval.comparison.load - INFO - Loading offers_snapshot
2025-07-16 10:57:

KeyboardInterrupt: 

In [24]:
datasets_processed

,quantity,revenue,retail_price,offer_value,discounts_value,Australia Day,Christmas Day,Good Friday,New Year's Day,consumer_sentiment_feat_centered,cpi_trimmed_mean_feat_smoothed,foot_traffic_vol,foot_traffic_vol_feat_weekly_pct_chng_smoothed
date,,,,,,,,,,,,,
2019-12-30,50401.0000,775599.1900,8.2211,0.0000,514.8000,0.0000,0.0000,0.0000,1.0000,-0.5900,0.0010,48033.0000,-0.0925
2020-01-06,73823.0000,1080639.3650,8.6603,0.0000,867.6800,0.0000,0.0000,0.0000,0.0000,-0.6114,0.0010,57194.5000,-0.0358
2020-01-13,73141.0000,1074458.4866,8.4185,0.0000,859.2400,0.0000,0.0000,0.0000,0.0000,-0.6364,0.0010,60053.0000,-0.0078
2020-01-20,74204.0000,1082677.7583,8.4341,0.0000,827.9200,1.0000,0.0000,0.0000,0.0000,-0.6613,0.0010,64027.0000,0.0661
2020-01-27,75712.0000,1119818.7200,8.5527,0.0000,811.2900,0.0000,0.0000,0.0000,0.0000,-0.6834,0.0010,51309.0000,0.0271
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-21,85247.0000,1718480.7700,9.9997,0.0000,11130.7300,0.0000,0.0000,0.0000,0.0000,-0.8888,0.0010,25523.5000,-0.0935
2025-04-28,87259.0000,1674810.9600,10.2538,0.0000,22151.0400,0.0000,0.0000,0.0000,0.0000,-0.9098,0.0010,31476.0000,-0.0088
2025-05-05,85661.0000,1663532.0300,10.2308,0.0000,26937.4900,0.0000,0.0000,0.0000,0.0000,-0.9244,0.0010,33582.5000,0.0171


In [ ]:
from mmm_eval.comparison.dataset_processor import DatasetProcessor

dp = DatasetProcessor.from_raw_data(datasets_processed, customer_id, data_version, holidays_whitelist,
                                    pipeline_path, node_filter=node_filter)

## Construct PyMC dataset and config

In [ ]:
# import os

# os.listdir("../../comparison")

In [25]:
datasets_processed.to_parquet("../../comparison/data/chicken_treat_consideration_at_store.parquet")

In [ ]:
#pymc_dataset = dp.get_pymc_dataset()

In [ ]:
from pymc_marketing.mmm import GeometricAdstock, LogisticSaturation
from mmm_eval.adapters.schemas import PyMCFitSchema, PyMCModelSchema
from mmm_eval import PyMCConfig, run_evaluation

col_map = dp.get_pymc_column_map()
#print(col_map.keys())

fast_fit_config = PyMCFitSchema(draws=1000, tune=1000, chains=4, target_accept=0.95, random_seed=42)
model_config = PyMCModelSchema(date_column="date", channel_columns=col_map["channel_columns"],
                               control_columns=col_map["control_columns"],
                               adstock=GeometricAdstock(l_max=8), saturation=LogisticSaturation(),
                               time_varying_intercept=False, time_varying_media=False,
                               yearly_seasonality=2)
config = PyMCConfig(pymc_model_config=model_config, fit_config=fast_fit_config, response_column="quantity",
                    revenue_column="revenue")

### Run evaluation doesn't work - trying to debug why

In [ ]:
# import pymc_marketing
# pymc_marketing.__file__

In [ ]:
# import importlib

# # Reload the deepest dependency first
# import pymc_marketing.model_builder
# importlib.reload(pymc_marketing.model_builder)

# # Reload the module that uses model_builder
# import pymc_marketing.mmm
# importlib.reload(pymc_marketing.mmm)

# # Reload your adapter module
# import mmm_eval.adapters.pymc
# importlib.reload(mmm_eval.adapters.pymc)

# # Now re-import PyMCAdapter from the reloaded module
# from mmm_eval.adapters.pymc import PyMCAdapter

# # Recreate adapter
# adapter = PyMCAdapter(config)

# adapter.fit(pymc_dataset.rename(columns={"quantity": "response"}))
# adapter.predict(pymc_dataset.rename(columns={"quantity": "response"}))

In [ ]:
#pymc_dataset.columns

                                                               
#result = run_evaluation(framework="pymc_marketing", config=config, data=pymc_dataset)

## Construct Meridian dataset and config

In [ ]:
col_map = dp.get_meridian_column_map()
col_map

In [ ]:
from mmm_eval.adapters.schemas import MeridianModelSpecSchema, MeridianSamplePosteriorSchema, MeridianInputDataBuilderSchema
from meridian import constants
import tensorflow_probability as tfp
from meridian.model import prior_distribution

roi_mu = 0.2     # Mu for ROI prior for each media channel.
roi_sigma = 0.9  # Sigma for ROI prior for each media channel.
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)

# set up Meridian configs
model_spec_config = MeridianModelSpecSchema(prior=prior)
sample_posterior_config = MeridianSamplePosteriorSchema(n_chains=4, n_adapt=200, n_burnin=200, n_keep=400)
idb_config = MeridianInputDataBuilderSchema(date_column="date", media_channels=col_map["media_channels"],
                                            channel_spend_columns=col_map["media_channels"],
                                            non_media_treatment_columns=col_map.get("non_media_treatment_columns"),
                                            control_columns=col_map.get("control_columns"),
                                            response_column="quantity")

In [ ]:
from mmm_eval.configs.configs import MeridianConfig

meridian_config = MeridianConfig(input_data_builder_config=idb_config, model_spec_config=model_spec_config,
                                 sample_posterior_config=sample_posterior_config, response_column="quantity",
                                 revenue_column="revenue")

In [ ]:
meridian_dataset = dp.get_meridian_dataset()

In [ ]:
result = run_evaluation(framework="meridian", config=meridian_config, data=meridian_dataset,
                        test_names=["accuracy"])

In [ ]:
result

In [ ]:
#meridian_dataset

ImportError: cannot import name 'storage' from 'google.cloud' (unknown location)

## Hypothesis: Meridian is somehow using revenue as the response column